# Sentiment Analysis

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf

raw_train_set, raw_valid_set, raw_test_set = tfds.load(
    name = 'imdb_reviews',
    split = ['train[:90%]', 'train[90%:]', 'test'],
    as_supervised= True
)

tf.random.set_seed(42)
train_set = raw_train_set.shuffle(5000, seed=42).batch(32).prefetch(1)
valid_set = raw_valid_set.batch(32).prefetch(1)
test_set = raw_test_set.batch(32).prefetch(1)

In [ ]:
for review, label in raw_train_set.take(6):
  print(review.numpy().decode('utf-8')[:250], '...')
  print('Label:', label.numpy())

In [ ]:
vocab_size = 1000
text_vec_layer = tf.keras.layers.TextVectorization(max_tokens=vocab_size,
                                                   standardize='lower_and_strip_punctuation')
text_vec_layer.adapt(train_set.map(lambda review, label:review))

In [ ]:
text_vec_layer.get_vocabulary()[:50]

In [ ]:
list(map(str, text_vec_layer.get_vocabulary()[-50:]))

In [ ]:
text_vec_layer(['i like it'])

In [ ]:
embed_layer = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=64)
embed_layer(text_vec_layer(['it was a great movie']))

In [ ]:
embed_size = 128
model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_size, mask_zero=True),
    tf.keras.layers.GRU(128),
    tf.keras.layers.Dense(1, activation='sigmoid')

])

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Nadam(learning_rate=1e-3),
              metrics = ['accuracy'])
model.fit(train_set, validation_data=valid_set, epochs=2)

In [ ]:
embedding_weights = model.layers[1].get_weights()[0]

In [ ]:
import numpy as np
np.savetxt('embedding.tsv', embedding_weights, delimiter='\t')

In [ ]:
vocab = text_vec_layer.get_vocabulary()

with open('metadata.tsv', 'w', encoding='utf-8') as f:
  for word in vocab:
    word = word if word.strip() != '' else '<PAD>'
    f.write(f'{word}\n')

# Generating Shakespearean Text Using a Character RNN

In [ ]:
import tensorflow as tf

shakespeare_url =  "https://homl.info/shakespeare"
filepath = tf.keras.utils.get_file('shakespeare.txt', shakespeare_url)
with open(filepath) as f:
  shakespeare_text = f.read()

In [ ]:
print(shakespeare_text[:100])

In [ ]:
''.join(sorted(set(shakespeare_text.lower())))

In [ ]:
text_vec_layer = tf.keras.layers.TextVectorization(split = 'character', standardize = 'lower')
text_vec_layer.adapt(shakespeare_text)
encoded = text_vec_layer([shakespeare_text][0])

In [ ]:
encoded

In [ ]:
encoded -= 2
n_tokens = text_vec_layer.vocabulary_size()-2
dataset_size = len(encoded)

In [ ]:
n_tokens

In [ ]:
dataset_size

In [ ]:
#input -> Before we proceed any furthe
#output -> efore we proceed any further

In [ ]:
def to_dataset(sequence , lenght ,seed =None,shuffle = False,batch_size = 32):
  ds = tf.data.Dataset.from_tensor_slices(sequence)
  ds = ds.window(lenght+1,shift=1,drop_remainder = True)
  ds = ds.flat_map(lambda window_ds: window_ds.batch(lenght+1))
  if shuffle :
    ds = ds.shuffle(100_000,seed=seed)
  ds = ds.batch(batch_size)
  return ds.map(lambda window:(window[:,:-1],window[:,1:]))

In [ ]:
lenght = 100
tf.random.set_seed(42)
train_set = to_dataset(encoded[:1_000_000],lenght=lenght,shuffle=True,
                       seed=42)
valid_set = to_dataset(encoded[1_000_000:1_060_000],lenght=lenght)
test_set = to_dataset(encoded[1_060_000:],lenght=lenght)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim = vocab_size,output_dim=16),
    tf.keras.layers.GRU(128,return_sequences = True),
    tf.keras.layers.Dense(vocab_size,activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",optimizer="nadam",
              metrics = ["accuracy"])

model_ckpt = tf.keras.callbacks.ModelCheckpoint(
    "my_shakespeare_model.keras",
    monitor="val_accuracy",
    save_best_only = True)

history = model.fit(train_set,validation_data=valid_set,epochs=1,
                    callbacks=[model_ckpt])

In [ ]:
shakespeare_model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Lambda(lambda X:X-2),
    model
])

In [ ]:
y_proba = shakespeare_model.predict(tf.constant(["To be or not to b"]))[0,-1]
y_pred = tf.argmax(y_proba)
text_vec_layer.get_vocabulary()[y_pred+2]

#Generating Fake Shakespeare text

In [ ]:
log_probas = tf.math.log([[0.5,0.3,0.2]])
tf.random.categorical(log_probas,num_samples=8)

In [ ]:
def next_char(text,temperature):
  text = tf.constant([text])
  y_proba = shakespeare_model.predict(text)[0,-1:]
  rescaled_logits = tf.math.log(y_proba)/temperature
  char_id = tf.random.categorical(rescaled_logits,num_samples=1)[0,0]
  return text_vec_layer.get_vocabulary()[char_id+2]

In [ ]:
def extent_text(text,chars=50,temperature = 1):
  for _ in range(chars):
    text +=next_char(text,temperature)
  return text

In [ ]:
extent_text('to be or not to be',chars = 100,temperature = 1)

#An Encoder-Decoder Network for Neural Machine Translation

In [ ]:
from pathlib import Path
import tensorflow as tf

url = "https://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip"
path = tf.keras.utils.get_file("spa-eng.zip", origin=url, cache_dir="datasets", extract=True)

# Final corrected path
spa_txt_path = Path(path).parent / "spa-eng_extracted" / "spa-eng" / "spa.txt"

# Read the file
text = spa_txt_path.read_text(encoding='utf-8')
print(text[:500])  # Print first 500 characters as a quick check

In [ ]:
import numpy as np

text = text.replace("i","").replace("¿","")
pairs = [line.split('\t')for line in text.splitlines()]
np.random.seed(42)
np.random.shuffle(pairs)
sentences_en ,sentences_es = zip(*pairs)

In [ ]:
pairs

In [ ]:
for i in range(3):
  print(sentences_en[i],"=>",sentences_es[i])

In [ ]:
vocab_size = 10000
max_lenght = 50

text_vec_layer_en = tf.keras.layers.TextVectorization(vocab_size,
                                                      output_sequence_length = max_lenght)

text_vec_layer_es = tf.keras.layers.TextVectorization(vocab_size,
                                                      output_sequence_length = max_lenght)

text_vec_layer_en.adapt(sentences_en)
text_vec_layer_es.adapt([f'startofseq {s} endofseq' for s in sentences_es])

In [ ]:
text_vec_layer_en.get_vocabulary()[:10]

In [ ]:
text_vec_layer_es.get_vocabulary()[:10]

In [ ]:
X_train = tf.constant(sentences_en[:100_000])
X_valid = tf.constant(sentences_es[100_000:])
X_train_dec = tf.constant([f'startofseq {s}' for s in sentences_es[:100_000]])
X_valid_dec = tf.constant([f'startofseq {s}' for s in sentences_es[100_000:]])
Y_train = text_vec_layer_es([f'{s} endofseq' for s in sentences_es[:100_000]])
Y_valid = text_vec_layer_es([f'{s} endofseq' for s in sentences_es[100_000:]])

In [ ]:
tf.random.set_seed(42)
encoder_inputs = tf.keras.layers.Input(shape=[], dtype=tf.string)
decoder_inputs = tf.keras.layers.Input(shape=[], dtype=tf.string)

In [ ]:
embed_size = 128
encoder_input_ids = text_vec_layer_en(encoder_inputs)
decoder_input_ids = text_vec_layer_es(decoder_inputs)
encoder_embedding_layer = tf.keras.layers.Embedding(vocab_size, embed_size,
                                                    mask_zero=True)
decoder_embedding_layer = tf.keras.layers.Embedding(vocab_size, embed_size, mask_zero=True)

encoder_embeddings = encoder_embedding_layer(encoder_input_ids)
decoder_embeddings = decoder_embedding_layer(decoder_input_ids)


In [ ]:
encoder = tf.keras.layers.LSTM(512, return_state=True)    #son vectoru qaytarsin deye state
encoder_outputs, *encoder_state = encoder(encoder_embeddings)


decoder = tf.keras.layers.LSTM(512, return_sequences=True)   #multioutput kimi her defe output versin deye sequence
decoder_outputs = decoder(decoder_embeddings, initial_state=encoder_state)

In [ ]:
output_layer = tf.keras.layers.Dense(vocab_size, activation='softmax')
Y_proba = output_layer(decoder_outputs)

In [ ]:
model = tf.keras.Model(inputs = [encoder_inputs, decoder_inputs],
                       outputs = [Y_proba])

model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = 'nadam',
              metrics = ['accuracy'])

# model.fit((X_train, X_train_dec), Y_train, epochs=3,
#           validation_data=[(X_valid, X_valid_dec), Y_valid])

In [ ]:
def translate(sentence_en):
  translation = ''
  for word_idx in range(max_lenght):
    X = tf.constant([sentence_en])
    X_dec = tf.constant(['startofseq' + translation])
    y_proba = model.predict((X,X_dec))[0, word_idx]
    predicted_word_id = np.argmax(y_proba)
    predicted_word = text_vec_layer_es.get_vocabulary()[predicted_word_id]
    if predicted_word == 'endofseq':
      break
    translation += ' ' + predicted_word
  return translation.strip()

In [ ]:
# translate('i love soccer')

# Bidirectional RNN

In [ ]:
tf.random.set_seed(42)
encoder = tf.keras.layers.Bidirectional(
    tf.keras.layers.LSTM(256, return_state = True)
)

In [ ]:
class ConcatenateStates(tf.keras.layers.Layer):
  def __init__(self):
    super().__init__()
  def call(self, encoder_state):
    return [tf.concat(encoder_state[::2], axis=-1),  #short_term(2,4,6)
              tf.concat(encoder_state[1::2],axis=-1)]   #long_term(1,3,5)

encoder_outputs, *encoder_state = encoder(encoder_embeddings)
concat_states = ConcatenateStates()
encoder_state = concat_states(encoder_state)

In [ ]:
decoder = tf.keras.layers.LSTM(512, return_sequences = True)
decoder_outputs = decoder(decoder_embeddings, initial_state=encoder_state)

output_layer = tf.keras.layers.Dense(vocab_size, activation='softmax')
Y_proba = output_layer(decoder_outputs)

model = tf.keras.Model(inputs = [encoder_inputs, decoder_inputs], outputs=[Y_proba])

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='nadam',
              metrics=['accuracy'])

# model.fit((X_train, X_train_dec), Y_train, epochs=3,
#           validation_data = [(X_valid, X_valid_dec), Y_valid])

In [ ]:
# translate('I like soccer and also going to the beach')

# Beam Search

In [ ]:
def beam_search(sentence_en, beam_width, verbose=False):
    X = tf.constant([sentence_en])  # encoder input
    X_dec = tf.constant(["startofseq"])  # decoder input
    y_proba = model.predict((X, X_dec))[0, 0]  # first token's probas
    top_k = tf.math.top_k(y_proba, k=beam_width)
    top_translations = [  # list of best (log_proba, translation)
        (np.log(word_proba), text_vec_layer_es.get_vocabulary()[word_id])
        for word_proba, word_id in zip(top_k.values, top_k.indices)
    ]

    # extra code – displays the top first words in verbose mode
    if verbose:
        print("Top first words:", top_translations)

    for idx in range(1, max_lenght):
        candidates = []
        for log_proba, translation in top_translations:
            if translation.endswith("endofseq"):
                candidates.append((log_proba, translation))
                continue  # translation is finished, so don't try to extend it
            X = tf.constant([sentence_en])  # encoder input
            X_dec = tf.constant(["startofseq " + translation])  # decoder input
            y_proba = model.predict((X, X_dec))[0, idx]  # last token's proba
            for word_id, word_proba in enumerate(y_proba):
                word = text_vec_layer_es.get_vocabulary()[word_id]
                candidates.append((log_proba + np.log(word_proba),
                                   f"{translation} {word}"))
        top_translations = sorted(candidates, reverse=True)[:beam_width]

        # extra code – displays the top translation so far in verbose mode
        if verbose:
            print("Top translations so far:", top_translations)

        if all([tr.endswith("endofseq") for _, tr in top_translations]):
            return top_translations[0][1].replace("endofseq", "").strip()

In [ ]:
sentence_en = 'I love cats and dogs'
translate(sentence_en)

In [ ]:
# beam_search(sentence_en, beam_width=3, verbose=True)

# Attention Mechanisms

In [ ]:
tf.random.set_seed(42)
encoder_inputs = tf.keras.layers.Input(shape=[], dtype=tf.string)
decoder_inputs = tf.keras.layers.Input(shape=[], dtype=tf.string)

In [ ]:
embed_size =128

encoder_inputs_idx = text_vec_layer_en(encoder_inputs)
decoder_inputs_idx = text_vec_layer_es(decoder_inputs)

encoder_embeddings_layer = tf.keras.layers.Embedding(vocab_size, output_dim=embed_size)
decoder_embeddings_layer = tf.keras.layers.Embedding(vocab_size, output_dim=embed_size)

encoder_embeddings = encoder_embeddings_layer(encoder_inputs_idx)
decoder_embeddings = decoder_embeddings_layer(decoder_inputs_idx)
# maskla attention error verir mask hissesini silirik

In [ ]:
encoder = tf.keras.layers.Bidirectional(
    tf.keras.layers.LSTM(256, return_sequences=True, return_state=True)
)

encoder_outputs, *encoder_state = encoder(encoder_embeddings)
encoder_state = concat_states(encoder_state)

In [ ]:
decoder = tf.keras.layers.LSTM(512, return_sequences=True)
decoder_outputs = decoder(decoder_embeddings, initial_state=encoder_state)

In [ ]:
attention_layer = tf.keras.layers.Attention()
attention_outputs = attention_layer([decoder_outputs, encoder_outputs])
output_layer = tf.keras.layers.Dense(vocab_size, activation='softmax')
Y_proba = output_layer(attention_outputs)

In [ ]:
model = tf.keras.Model(inputs = [encoder_inputs, decoder_inputs], outputs = [Y_proba])

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='nadam',
              metrics = ['accuracy'])

model.fit((X_train, X_train_dec), Y_train, epochs=3, validation_data=[(X_valid, X_valid_dec), Y_valid])

In [ ]:
translate('I like soccer and also going to the beach')

# Transformer model

In [ ]:
import tensorflow as tf

vocab_size = 10000
max_length = 50
embed_size = 128
num_heads = 5
ff_dim = 512


# input layers
encoder_inputs = tf.keras.Input(shape=(None,), dtype=tf.int32, name='encoder_inputs')
decoder_inputs = tf.keras.Input(shape=(None,), dtype=tf.int32, name='decoder_inputs')

# embedding layer

encoder_embedding_layer = tf.keras.layers.Embedding(vocab_size,
                                                    output_dim=embed_size,
                                                    mask_zero=True)
decoder_embedding_layer = tf.keras.layers.Embedding(vocab_size,
                                                    output_dim=embed_size,
                                                    mask_zero=True)

encoder_embeddings = encoder_embedding_layer(encoder_inputs)
decoder_embeddings = decoder_embedding_layer(decoder_inputs)

# positional embedding

pos_embedding_layer = tf.keras.layers.Embedding(max_length, embed_size)
positions_encoder = tf.keras.layers.Lambda(lambda x: tf.range(start=0, limit=tf.shape(x)[1],
                                                              delta = 1))(encoder_inputs)
positions_decoder = tf.keras.layers.Lambda(lambda x: tf.range(start=0, limit=tf.shape(x)[1],
                                                              delta = 1))(decoder_inputs)

pos_embed_enc = pos_embedding_layer(positions_encoder)
pos_embed_dec = pos_embedding_layer(positions_decoder)

# Add tokens and postional embeddings
encoder_embed = encoder_embeddings + pos_embed_enc
decoder_embed = decoder_embeddings + pos_embed_dec

In [ ]:
#  Encoder self Attention
encoder_attention = tf.keras.layers.MultiHeadAttention(num_heads=num_heads,
                                                       key_dim = embed_size)(encoder_embed, encoder_embed)  #query, key encoder_embed olur
encoder_attention = tf.keras.layers.LayerNormalization(epsilon = 1e-6)(encoder_embed + encoder_attention)


# Encoder Feed Forward
encoder_ff = tf.keras.layers.Dense(ff_dim, activation='relu')(encoder_attention)
encoder_ff = tf.keras.layers.Dense(embed_size)(encoder_ff)
encoder_outputs = tf.keras.layers.LayerNormalization(epsilon=(1e-6))(encoder_attention + encoder_ff)

In [ ]:
# Decoder self attention
casual_mask = tf.keras.layers.Lambda(lambda x: tf.linalg.band_part(tf.ones((tf.shape(x)[1], tf.shape(x)[1])), -1, 0))(decoder_inputs)
decoder_attention = tf.keras.layers.MultiHeadAttention(num_heads=num_heads,
                                                       key_dim = embed_size)(decoder_embed, decoder_embed, attention_mask = casual_mask)
decoder_attention = tf.keras.layers.LayerNormalization(epsilon=1e-6)(decoder_embed+decoder_attention)

In [ ]:
# Decoder Encoder cross attetion
cross_attention = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_size)(decoder_attention,
                                                                                              encoder_outputs, encoder_outputs)
decoder_cross = tf.keras.layers.LayerNormalization(epsilon=1e-6)(decoder_attention+cross_attention)

In [ ]:
# Decoder Feed forward
decoder_ff = tf.keras.layers.Dense(ff_dim, activation='relu')(decoder_cross)
decoder_ff = tf.keras.layers.Dense(embed_size)(decoder_ff)
decoder_outputs = tf.keras.layers.LayerNormalization(epsilon=1e-6)(decoder_cross+decoder_ff)

In [ ]:
# Finel Output Layer
output_logits = tf.keras.layers.Dense(vocab_size, activation='softmax')(decoder_outputs)
transformer = tf.keras.Model([encoder_inputs, decoder_inputs], output_logits)

In [ ]:
transformer.compile(loss='sparse_categorical_crossentropy',
                    optimizer='nadam',
                    metrics =['accuracy'])

# transformer.fit((X_train, X_train_dec), Y_train, epochs=3, validation_data = ((X_valid, X_valid_dec), Y_valid))

In [ ]:
vocab_size = 15000
max_lenght = 50

text_vec_layer_en = tf.keras.layers.TextVectorization(vocab_size,
                                                      output_sequence_length = max_lenght,
                                                      pad_to_max_tokens=True)

text_vec_layer_es = tf.keras.layers.TextVectorization(vocab_size,
                                                      output_sequence_length = max_lenght,
                                                      pad_to_max_tokens = True)

text_vec_layer_en.adapt(sentences_en)
text_vec_layer_es.adapt([f'startofseq {s} endofseq' for s in sentences_es])

In [ ]:
X_train_padded = tf.keras.preprocessing.sequence.pad_sequences(text_vec_layer_en(X_train).numpy(),
                                                               padding='post', maxlen=max_lenght)
X_train_dec_padded =tf.keras.preprocessing.sequence.pad_sequences(text_vec_layer_es(X_train_dec).numpy(),
                                                                  padding='post', maxlen=max_lenght)
X_valid_padded = tf.keras.preprocessing.sequence.pad_sequences(text_vec_layer_en(X_valid).numpy(),
                                                               padding='post', maxlen=max_lenght)
X_valid_dec_padded = tf.keras.preprocessing.sequence.pad_sequences(text_vec_layer_es(X_valid_dec).numpy(),
                                                                   padding='post', maxlen=max_lenght)

X_train_padded = tf.constant(X_train_padded)
X_train_dec_padded = tf.constant(X_train_dec_padded)
X_valid_padded = tf.constant(X_valid_padded)
X_valid_dec_padden = tf.constant(X_valid_dec_padded)

transformer.fit((X_train_padded, X_train_dec_padded), Y_train, epochs=3,
                validation_data = ((X_valid_padded, X_valid_dec_padded), Y_valid))

In [ ]:
import numpy as np

def translate(sentence_en):
    # Tokenize and pad encoder input
    X = text_vec_layer_en(tf.constant([sentence_en]))
    X = tf.keras.preprocessing.sequence.pad_sequences(X.numpy(), padding="post", maxlen=max_length)

    # Start token
    start_token = text_vec_layer_es(['startofseq'])[0][0]
    end_token = text_vec_layer_es(['endofseq'])[0][0]

    # Decoder input initialized with just the start token
    decoder_input = [start_token]

    for _ in range(max_length):
        decoder_input_padded = tf.keras.preprocessing.sequence.pad_sequences(
            [decoder_input], maxlen=max_length, padding="post"
        )

        y_proba = transformer.predict((X, decoder_input_padded), verbose=0)[0, len(decoder_input)-1]
        predicted_word_id = np.argmax(y_proba)

        if predicted_word_id == end_token:
            break

        decoder_input.append(predicted_word_id)

    # Map tokens back to words
    vocab = text_vec_layer_es.get_vocabulary()
    translated_words = [vocab[token] for token in decoder_input[1:]]  # skip start token

    return ' '.join(translated_words)

In [ ]:
print(translate('i like soccer and also going to the beach'))